In [74]:
#!pip install langchain_nebius


In [75]:
#!pip install pygraphviz

In [ ]:
import os 
import json
from typing import List, TypedDict, Optional
from dotenv import load_dotenv
from google.colab import files
from langchain_openai import ChatOpenAI

from langchain_nebius import ChatNebius
from pydantic import BaseModel,Field 
from langgraph.graph import StateGraph, END

# for p\Pretty Printing
from rich.console import Console
from rich.markdown import Markdown
from rich.syntax import Syntax

#-- API Key and Tracing Setup--
#from dotenv import load_dotenv
#load_dotenv('.env')

os.environ['LANGCHAIN_API_KEY'] = ''

# CRITICAL: Map NEBIUS_API_KEY to OPENAI_API_KEY for ChatNebius
# ChatNebius requires OPENAI_API_KEY, not NEBIUS_API_KEY
os.environ['OPENAI_API_KEY'] = ''

#Setup langsmith tracing
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="Agentic Architecture - Reflection(Nebius)"

# Check that keys are set
if not os.environ.get('NEBIUS_API_KEY'):
    print('NEBIUS API KEY NOT FOUND')
if not os.environ.get('LANGCHAIN_API_KEY'):
    print('Langchain API Key missing')

print('Environment variables loaded and traching is steup ')

Environment variables loaded and traching is steup 


Defining the Data Schemas with Pydantic


In [77]:
class DraftCode(BaseModel):
    """Schema for the initial code draft generated by the agent."""
    code:str=Field(description="The python code generated to solve the user request")
    explanation:str=Field(description="A brief explanation of the code and how it works")

class Critique(BaseModel):
    has_erros:bool=Field(description="Does the code have any potential bugs or logical errors?")
    is_efficient: bool=Field(description="Is the code written in an efficient and optimal way?")
    suggested_improvements:List[str]=Field(description="Specific, actionable suggestions for improving the code")
    critique_summary:str=Field(description="A concise summary of the overall critique")

class RefinedCode(BaseModel):
    refined_code:str=Field(description="The final, improved Python code")
    refinement_summary:str=Field(description="A summary of the changes made to the original code based on the critique")

print("Pydantic models for draft, critiquw, and refinedCode have been defined")
    



Pydantic models for draft, critiquw, and refinedCode have been defined


Initializing the Nebius LLm and the COnsole


In [78]:
#Use a powerfull Nebius model fro generation and critique
#Use OpenAI model for generation and critique
llm = ChatOpenAI(
    model='gpt-4o-mini',  # Fast and cheap: 'gpt-4o-mini' or 'gpt-3.5-turbo'
                          # More powerful: 'gpt-4o' or 'gpt-4'
    temperature=0.2
)

#Initialize console for pretty printing
console = Console()

print('OpenAI LLM and Console are initialized')


OpenAI LLM and Console are initialized


In [79]:
#CREATING THE GENERATOR CODE
def generator_node(state):
    """Generates the initial draft of the code"""
    console.print("-- 1. Genrating Initial Draft--")
    generator_llm=llm.with_structured_output(DraftCode)

    prompt=f"""You are an expert Python programmer. Write a Python function to solve the following request.
    Provide a simple, clear implementation and an explanation.

    Request: {state['user_request']}"""

    draft=generator_llm.invoke(prompt)
    return {"draft":draft.model_dump()} # corrected: use .model_dump


##Creating the critic node


In [80]:
def critic_node(state):
    """Critiques the generated code for errors and inefficiencies"""
    console.print("--2. Critique Draft--  ")
    critic_llm=llm.with_structured_output(Critique)

    code_to_critique=state['draft']['code']

    prompt=f"""You are an expert code reviewer and senior Python developer. Your task is to perform a thorough critique of the following code.
    
    Analyze the code for:
    1.  **Bugs and Errors:** Are there any potential runtime errors, logical flaws, or edge cases that are not handled?
    2.  **Efficiency and Best Practices:** Is this the most efficient way to solve the problem? Does it follow standard Python conventions (PEP 8)?
    
    Provide a structured critique with specific, actionable suggestions.
    
    Code to Review:```python
    {code_to_critique}
    ```
    
    """

    critique=critic_llm.invoke(prompt)
    return {'critique':critique.model_dump()}

## Refiner Node

In [81]:
def refiner_node(state):
    """Refines the code based on the critique."""
    console.print('---3. Refining Code')
    refiner_llm=llm.with_structured_output(RefinedCode)

    draft_code=state['draft']['code']
    critique_suggestions=json.dumps(state['critique'],indent=2)

    prompt=f"""You are an expert Python programmer tasked with refining a piece of code based on a critique.
    
    Your goal is to rewrite the original code, implementing all the suggested improvements from the critique.
    
    **Original Code:**
    ```python
    {draft_code}
    ```
    
    **Critique and Suggestions:**
    {critique_suggestions}
    
    Please provide the final, refined code and a summary of the changes you made.
    """

    refined_code=refiner_llm.invoke(prompt)
    return {'refined_code':refined_code.model_dump()}



Phase 2 Orchestration:

In [82]:
class Reflection_state(TypedDict):
    """Represents the statue of our reflection graph"""
    user_request:str
    draft: Optional[dict]
    critique: Optional[dict]
    refined_code: Optional[dict]

print('ReflectionState TypedDict defined')

ReflectionState TypedDict defined


In [83]:
graph_builder=StateGraph(Reflection_state)

#Add thenodes to the graph
graph_builder.add_node("generator",generator_node)
graph_builder.add_node("critic",critic_node)
graph_builder.add_node("refiner",refiner_node)

#Define the workflow edges
graph_builder.set_entry_point("generator")
graph_builder.add_edge("generator","critic")
graph_builder.add_edge("critic","refiner")
graph_builder.add_edge("refiner",END)

#Compile the graph
reflection_app=graph_builder.compile()

print("Reflection graph compile Successfullyyy")

#Visualize the graph
try:
    from IPython.display import Image, display
    png_image=reflection_app.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}. Please ensure pygraphviz is installed..")

Reflection graph compile Successfullyyy
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed..


In [84]:
user_request="Write a Pythin function to find thenth fibonacci number"
initial_input={"user_request":user_request}

console.print(f"[bold cyan] Kicking odd Reflection workflow for request:[/bold cyan]{user_request}")

#The loop captures the fully populated state
final_state=None
for state_update in reflection_app.stream(initial_input,stream_mode="values"):
    final_state=state_update

console.print("[bold green]---Reflection Workflow complete ---[/bold green]" )

 Kicking odd Reflection workflow for request:Write a Pythin function to find thenth fibonacci number

-- 1. Genrating Initial Draft--

--2. Critique Draft--

---3. Refining Code

---Reflection Workflow complete ---

In [85]:
# Check if final_state is available and has the expected keys
if final_state and 'draft' in final_state and 'critique' in final_state and 'refined_code' in final_state:
    console.print(Markdown("--- ### Initial Draft ---"))
    console.print(Markdown(f"**Explanation:** {final_state['draft']['explanation']}"))
    # Use rich's Syntax for proper code highlighting
    console.print(Syntax(final_state['draft']['code'], "python", theme="monokai", line_numbers=True))

    console.print(Markdown("\n--- ### Critique ---"))
    console.print(Markdown(f"**Summary:** {final_state['critique']['critique_summary']}"))
    console.print(Markdown(f"**Improvements Suggested:**"))
    for improvement in final_state['critique']['suggested_improvements']:
        console.print(Markdown(f"- {improvement}"))

    console.print(Markdown("\n--- ### Final Refined Code ---"))
    console.print(Markdown(f"**Refinement Summary:** {final_state['refined_code']['refinement_summary']}"))
    console.print(Syntax(final_state['refined_code']['refined_code'], "python", theme="monokai", line_numbers=True))
else:
    console.print("[bold red]Error: The `final_state` is not available or is incomplete. Please check the execution of the previous cells.[/bold red]")

--- ### Initial Draft ---

Explanation: The function fibonacci(n) calculates the nth Fibonacci number. It first checks if the input n is a    
positive integer. If n is 1, it returns 0 (the first Fibonacci number), and if n is 2, it returns 1 (the second    
Fibonacci number). For values of n greater than 2, it uses a loop to calculate the Fibonacci numbers iteratively.  
It starts with the first two Fibonacci numbers (0 and 1) and updates them until it reaches the nth number, which it
then returns.

   1 def fibonacci(n):                                                                                             
   2     if n <= 0:                                                                                                
   3         return "Input should be a positive integer."                                                          
   4     elif n == 1:                                                                                              
   5         return 0                                                                                              
   6     elif n == 2:                                                                                              
   7         return 1                                                                                              
   8     else:                                                                                                     
   9         a, b = 0, 1                                                                                           
  10         for _ in range(2, n):                                                                                 
  11             a, b = b, a + b                                                                                   
  12         return b                                                                                              
  13                                                                                                               

--- ### Critique ---

Summary: The code has indentation issues and does not handle invalid input appropriately. It is efficient for small
values of n but could be optimized for larger inputs.

...

 • Consider using memoization or an iterative approach for larger values of n to improve efficiency.

 • Add type hints to the function signature for better clarity.

--- ### Final Refined Code ---

Refinement Summary: The code has been refined by fixing the indentation to follow Python standards (4 spaces per   
level). The return type for invalid input has been changed to raise a ValueError instead of returning a string.    
Type hints have been added to the function signature for better clarity. The iterative approach remains for        
efficiency with larger values of n.

   1 def fibonacci(n: int) -> int:                                                                                 
   2     if n <= 0:                                                                                                
   3         raise ValueError("Input should be a positive integer.")                                               
   4     elif n == 1:                                                                                              
   5         return 0                                                                                              
   6     elif n == 2:                                                                                              
   7         return 1                                                                                              
   8     else:                                                                                                     
   9         a, b = 0, 1                                                                                           
  10         for _ in range(2, n):                                                                                 
  11             a, b = b, a + b                                                                                   
  12         return b                                                                                              
  13                                                                                                               

## Quantitative Evaluation

In [86]:
class codeEvaluation(BaseModel):
    """Schema fro evaluating a piece of code"""
    correctness_score: int=Field(description="A score from 1 to 10 indicating how correct the code is logically correct")
    efficiency_score: int = Field(description="Score from 1-10 on the code's algorithmic efficiency.")
    style_score: int = Field(description="Score from 1-10 on code style and readability (PEP 8). ")
    justification: str = Field(description="A brief justification for the scores.")


judge_llm=llm.with_structured_output(codeEvaluation)

def evaluate_code(code_to_evaluate:str):
    prompt = f"""You are an expert judge of Python code. Evaluate the following function on a scale of 1-10 for correctness, efficiency, and style. Provide a brief justification.
    
    Code:
    ```python
    {code_to_evaluate}
    ```
    """
    return judge_llm.invoke(prompt)


if final_state and 'draft' in final_state and 'refined_code' in final_state:
    console.print("--- Evaluating Initial Draft ---")
    initial_draft_evaluation = evaluate_code(final_state['draft']['code'])
    console.print(initial_draft_evaluation.model_dump()) # Corrected: use .model_dump()

    console.print("\n--- Evaluating Refined Code ---")
    refined_code_evaluation = evaluate_code(final_state['refined_code']['refined_code'])
    console.print(refined_code_evaluation.model_dump()) # Corrected: use .model_dump()
else:
    console.print("[bold red]Error: Cannot perform evaluation because the `final_state` is incomplete.[/bold red]")


--- Evaluating Initial Draft ---

{
    'correctness_score': 8,
    'efficiency_score': 7,
    'style_score': 6,
    'justification': 'The function correctly computes the nth Fibonacci number for positive integers, but it 
returns a string for invalid input, which is not ideal for a function expected to return an integer. The efficiency
is decent with a linear time complexity, but it could be improved with memoization or a closed-form solution. The 
style lacks proper indentation and does not follow PEP 8 conventions for spacing and line length.'
}

--- Evaluating Refined Code ---

{
    'correctness_score': 7,
    'efficiency_score': 6,
    'style_score': 5,
    'justification': 'The function correctly calculates Fibonacci numbers for positive integers, but it returns 0 
for n=1 and 1 for n=2, which is inconsistent with the standard Fibonacci sequence where F(1)=1 and F(2)=1. The 
efficiency is reasonable with a linear time complexity, but it could be improved with memoization or a closed-form 
solution. The style lacks proper indentation and does not follow PEP 8 guidelines for spacing and line length.'
}